# Aula 12 - Herança Múltipla

Este documento apresenta como Python trabalha com herança múltipla.

## Exemplo 1: Herança sem Ambiguidade

O código a seguir apresenta uma classe que possui duas classes base. O código funciona como esperado, já que as classes base não possuem ambiguidade, isto é os seus métodos e atributos não possuem o mesmo nome.

In [ ]:
class Superclasse1:
    def __init__(self, valor):
        self.atrib_super1 = valor

    def metodo_super1(self):
        print('Metodo super1')

class Superclasse2:
    def __init__(self, valor):
        self.atrib_super2 = valor

    def metodo_super2(self):
        print('Metodo super2')

class Subclasse(Superclasse1, Superclasse2):
    def __init__(self, valor):
        Superclasse1.__init__(self, 0) # atribui 0 a atrib_super1
        Superclasse2.__init__(self, 1) # atribui 1 a atrib_super2
        self.atrib_sub = valor

    def metodo_sub(self):
        print('Metodo sub')

if __name__ == "__main__":
    obj = Subclasse(50)
    print(obj.atrib_super1)
    print(obj.atrib_super2)
    print(obj.atrib_sub)
    obj.metodo_super1()
    obj.metodo_super2()
    obj.metodo_sub()

## Exemplo 2 : Herança com Atributos e Métodos com Mesmo Nome

O código a seguir possui uma classe derivada que herda de duas classes base com métodos e atributos em comum. Observe quais dos atributos/métodos das classes base são herdados pela classe derivada.

In [ ]:
class Superclasse1:
    def __init__(self, valor):
        print('Inicializador de super1')
        self.atrib_super = valor

    def metodo_super(self):
        print('Metodo super de Superclasse1')

class Superclasse2:
    def __init__(self, valor):
        print('Inicializador de super2')
        self.atrib_super = valor

    def metodo_super(self):
        print('Metodo super de Superclasse2')

class Subclasse(Superclasse2, Superclasse1):
    def __init__(self, valor):
        Superclasse2.__init__(self, 1) # atribui 1 a atrib_super de Super2
        Superclasse1.__init__(self, 0) # atribui 0 a atrib_super de Super1
        self.atrib_sub = valor

    def metodo_sub(self):
        print('Metodo sub')

if __name__ == "__main__":
    obj = Subclasse(50)
    print(obj.atrib_super) # qual atrib_super e utilizado?
    obj.metodo_super() # qual metodo_super e chamado?

Note que  as classes ```Superclasse1``` e ```Superclasse2``` possuem
um atributo e um método com o mesmo nome:

- O método herdado pela ```Subclasse``` depende da ordem indicada
  na tupla de classes base:
    - A linguagem Python considera a primeira superclasse da esquerda para a direita
    - A implementação do método que for achada primeiro é utilizada
    - Caso uma implementação do método não seja achada em nenhuma das classes base, a busca é realizada recursivamente nas classes base das classes base
    - **Para saber a ordem de busca das classes:** utilize o método de classe `mro` (*method resolution order*) presente em todas as classes Python
- Em relação aos atributos:
    - Como o `__init__` de cada superclasse foi chamado no 
      `__init__` da subclasse, o atributo considerado é o último
      encontrado (e não o primeiro)
    - Cada chamada de `__init__` sobrescreve a declaração
      anterior
    - Portanto, o que vale é o último que sobrescreve

### Possível problema com atributos com nomes comuns

O código a seguir possui duas classes base que possuem um atributo com mesmo nome `x`. Observe o que pode acontecer neste caso.

In [ ]:
class A:
    def __init__(self): 
        self.x = 0

    def m1(self): 
        return self.x + 1

class B:
    def __init__(self): 
        self.x = []

class C(A,B):
    def __init__(self):
        A.__init__(self)
        B.__init__(self)

if __name__ == '__main__':
    c = C()
    print(c.m1()) # Funciona ? Qual é o problema?
    #print(C.mro()) # Imprime a ordem das classes buscadas para métodos da classe C

## Exemplo 3: O problema do Diamante 

Ao utilizar herança múltipla, problemas podem ocorrer com hierarquias em formato de diamante, como mostrado na figura a seguir.

![Diamante](https://raw.githubusercontent.com/ect-info/POO_2021.1/master/docs/12-heranca-multipla/diamante.png)

Especificamente, considere os casos listados a seguir.

### 1. Todas as classes implementam `metodo`

Qual versão de `metodo` será chamada para objetos da classe `D`?

In [ ]:
class A:
    def metodo(self):
        print('Metodo de A')

class B(A):
    def metodo(self):
        print('Metodo de B')

class C(A):
    def metodo(self):
        print('Metodo de C')

class D(B,C):
    def metodo(self):
        print('Metodo de D')

if __name__ == "__main__":
    a = A()
    b = B()
    c = C()
    d = D()

    a.metodo()
    b.metodo()
    c.metodo()
    d.metodo()

### 2. `D` apenas herda `metodo` (`metodo` não é sobrescrito)
   
Qual versão de `metodo` será chamada para objetos da classe `D`?

In [ ]:
class A:
    def metodo(self):
        print('Metodo de A')

class B(A):
    def metodo(self):
        print('Metodo de B')

class C(A):
    def metodo(self):
        print('Metodo de C')

class D(B,C):
    pass

if __name__ == "__main__":
    a = A()
    b = B()
    c = C()
    d = D()

    a.metodo()
    b.metodo()
    c.metodo()
    d.metodo()

### 3. Tanto `D` quanto `B` apenas herda `metodo`

Qual versão de `metodo` será chamada para objetos da classe `D`?

In [ ]:
class A:
    def metodo(self):
        print('Metodo de A')

class B(A):
    pass

class C(A):
    def metodo(self):
        print('Metodo de C')

class D(B,C):
    pass

if __name__ == "__main__":
    a = A()
    b = B()
    c = C()
    d = D()

    a.metodo()
    b.metodo()
    c.metodo()
    d.metodo()

Observe que no caso anterior, ao invés de `c.metodo()` realizar uma chamada ao `metodo` da classe `A` (que é imediatamente a sua superclasse), `c.metodo()` realiza uma chamada ao `metodo` da classe `B`.

A linguagem Python trabalha desta forma no que se chama de *herança múltipla cooperativa*. Assim, em hierarquias mais complexas, é garantido que a implementação de `metodo` "mais próxima" da classe original seja chamada, ao invés da implementação "mais ancestral".

Para mais informações sobre herança múltipla cooperativa em Python, veja este [artigo](https://www.artima.com/weblogs/viewpost.jsp?thread=281127).

### Chamando métodos comuns de uma hierarquia em uma determinada ordem

Observe o código a seguir e perceba como o uso de `super` pode ser útil para implementar um encadeamento de chamada de métodos em uma determinada ordem.

In [ ]:
class A:
    def metodo(self):
        print('Metodo de A')

class B(A):
    def metodo(self):
        super().metodo() # irá chamar c.metodo()
        print('Metodo de B')

class C(A):
    def metodo(self):
        super().metodo() # irá chamar a.metodo()
        print('Metodo de C')

class D(C,B):
    def metodo(self):
        super().metodo() # irá chamar b.metodo()
        print('Metodo de D')

if __name__ == "__main__":
    d = D()
    d.metodo() # apenas uma chamada -> encadeia chamadas ao mesmo método de toda a hierarquia

## Prática 2.4 - Processador de Texto

Implemente o sistema orientado a objetos exibido no diagrama de classe a seguir. Este sistema serve para processar textos, exibindo diferentes funcionalidades como contar quantas vezes ocorre cada palavra ou traduzir palavras do português para inglês (ignore a forma rudimentar :) )

![Diagrama de classes](https://raw.githubusercontent.com/ect-info/POO_2021.1/master/docs/12-heranca-multipla/pratica_heranca_multipla.png)

Na sua implementação, considere o seguinte:

- É obrigatório manter a interface pública de cada classe como indicada no diagrama de classes
- Cada classe deve pode ser instanciada e funcionar por si só
- Implemente métodos (não publicos) auxiliares, caso deseje
- Os atributos presentes no diagrama servem apenas como implementação de referência. Você está livre implementar cada classe como achar mais conveniente, desde que ela faça o que se pede:
    - Classe `PreProcessador`: o método `processa` deve transformar um texto em uma lista de palavras. Na implementação de referência, o resultado é armazenado no atributo `lista_palavras`
    - Classe `ContadorPalavras`: o método `processa` deve contar quantas vezes cada palavra ocorre em uma lista. Na implementação de referência, o resultado é armazenado no atributo `ocorrencias`
    - Classe `Tradutor`: o método `processa` deve traduzir palavras contidas em uma lista para inglês. Na implementação de referência, as traducoes estão no atributo `traducoes` e o resultado é armazenado no atributo `lista_palavras_trad`
    - Classe `ProcessadorTexto`: o método `processa` deve, utilizando as classes `ContadorPalavras` e `Tradutor` exibir uma tradução rudimentar do texto original em inglês em uma única string e a frequência das palavras do texto de entrada
    
Utilize o código a seguir para testar a sua implementação.

In [ ]:
if __name__ == '__main__':
    # Descomente a seguir para testar apenas a classe PreProcessador
    # preprocessador = PreProcessador('OLá! Este é um exemplo de texto com termos repetidos.'
    #                                 ' Este texto possui vários termos repetidos:'
    #                                 ' este, Este, ESte, esTE!')
    # preprocessador.processa()
    # print(preprocessador)
    
    # Descomente a seguir para testar apenas a classe ContadorPalavras
    # contador = ContadorPalavras('OLá! Este é um exemplo de texto com termos repetidos.'
    #                             ' Este texto possui vários termos repetidos:'
    #                             ' este, Este, ESte, esTE!')
    # contador.processa()
    # print(contador)

    # Descomente a seguir para testar apenas a classe Tradutor
    # tradutor = Tradutor('OLá! Este é um exemplo de texto com termos repetidos.'
    #                     ' Este texto possui vários termos repetidos:'
    #                     ' este, Este, ESte, esTE!')
    # tradutor.processa()
    # print(tradutor)

    processadortexto = ProcessadorTexto('OLá! Este é um exemplo de texto com termos repetidos.'
                                        ' Este texto possui vários termos repetidos:'
                                        ' este, Este, ESte, esTE!')
    processadortexto.processa()

Saída esperada:

```
Frequencia das palavras:
olá: 1 vezes
este: 6 vezes
é: 1 vezes
um: 1 vezes
exemplo: 1 vezes
de: 1 vezes
texto: 2 vezes
com: 1 vezes
termos: 2 vezes
repetidos: 2 vezes
possui: 1 vezes
vários: 1 vezes

Tradução robótica:
hello this is a example of text with terms repeated this text has various terms repeated this this this this
```